In [1]:
import os
import json
import pandas as pd
from glob import glob

In [2]:
base_dir = r'D:\Middleproject\수학분야 학습자 역량 측정 데이터\Training\1학년'
skill_eval_folders = [f'실력평가{str(i).zfill(3)}' for i in range(1, 193)]

In [3]:
# 각 폴더에서 JSON 파일을 읽어 리스트로 반환하는 함수
def read_json_files(folder_path):
    data = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    data.append(json.loads(line))
    return data

# 데이터 프레임을 병합하는 함수
def merge_data_from_folders(folders):
    all_data = []
    for folder in folders:
        skill_eval_folder = os.path.join(base_dir, folder)
        
        # 하위 폴더 경로 정의
        subfolders = {
            '1_Question Correct Answer Table': os.path.join(skill_eval_folder, '1_문항정오답표'),
            '2_Question IRT': os.path.join(skill_eval_folder, '2_문항IRT'),
            '3_Test-Taker IRT': os.path.join(skill_eval_folder, '3_응시자IRT')
        }
        
        # JSON 파일 읽기
        qa_data = read_json_files(subfolders['1_Question Correct Answer Table'])
        irt_data = read_json_files(subfolders['2_Question IRT'])
        taker_irt_data = read_json_files(subfolders['3_Test-Taker IRT'])
        
        # 데이터 프레임 생성
        qa_df = pd.DataFrame(qa_data)
        irt_df = pd.DataFrame(irt_data)
        taker_irt_df = pd.DataFrame(taker_irt_data)
        
        # 데이터 프레임을 learnerID와 testID로 병합
        merged_df = pd.merge(qa_df, irt_df, on=['testID', 'assessmentItemID'], how='left')
        merged_df = pd.merge(merged_df, taker_irt_df, on=['learnerID', 'testID'], how='left')
        
        all_data.append(merged_df)
    
    # 모든 데이터 프레임을 하나로 병합
    final_df = pd.concat(all_data, ignore_index=True)
    return final_df

In [4]:
# 모든 평가 폴더에서 데이터 읽기 및 병합
combined_df = merge_data_from_folders(skill_eval_folders)